In [1]:
from pyquil import Program, get_qc
from pyquil.paulis import *
import qulacs as q
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_file
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_text
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.transforms import get_sparse_operator
from qulacs.observable import create_split_observable, create_observable_from_openfermion_text
from qulacs import Observable, QuantumStateGpu, QuantumCircuit
import pyquil.api as api
import numpy as np
from numpy import linalg as LA
import math
import random
import itertools

In [35]:
model4site = [
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 2, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 2, "neighbors": [0,2], "hoppings": [1,0,1,0]}
]

model4site1d = [
    {"U": 0, "neighbors": [1],   "hoppings": [0,1,0,0]},
    {"U": 1, "neighbors": [0,2], "hoppings": [1,0,1,0]},
    {"U": 1, "neighbors": [1,3], "hoppings": [0,1,0,1]},
    {"U": 1, "neighbors": [2],   "hoppings": [0,0,1,0]}
]

model2site = [
    {"U": 2, "neighbors": [1], "hoppings": [0,1]},
    {"U": 2, "neighbors": [0], "hoppings": [1,0]}
]

model6site = [
    {"U": 2, "neighbors": [1,2,5], "hoppings": [0,1,1,0,0,1]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0]},
    {"U": 2, "neighbors": [0,1,3], "hoppings": [1,1,0,1,0,0]},
    {"U": 2, "neighbors": [2,4,5], "hoppings": [0,0,1,0,1,1]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1]},
    {"U": 2, "neighbors": [0,3,4], "hoppings": [1,0,0,1,1,0]}
]

model8site = [
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,1,0,1,0,0,0,1]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [1,0,1,0,0,0,1,0]},
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,1,0,1,0,1,0,0]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [1,0,1,0,1,0,0,0]},
    {"U": 2, "neighbors": [3,5,7], "hoppings": [0,0,0,1,0,1,0,1]},
    {"U": 2, "neighbors": [2,4,6], "hoppings": [0,0,1,0,1,0,1,0]},
    {"U": 2, "neighbors": [1,5,7], "hoppings": [0,1,0,0,0,1,0,1]},
    {"U": 2, "neighbors": [0,4,6], "hoppings": [1,0,0,0,1,0,1,0]}
]

t = t2 = t3 = 1
modelGutzwiller = [
    {"U": 2, "neighbors": [1,3,5], "hoppings": [0,t,0,t,0,t2,0,0]},
    {"U": 2, "neighbors": [0,2,6], "hoppings": [t,0,t,0,0,0,t2,0]},
    {"U": 2, "neighbors": [1,3,7], "hoppings": [0,t,0,t,0,0,0,t2]},
    {"U": 2, "neighbors": [0,2,4], "hoppings": [t,0,t,0,t2,0,0,0]},
    {"U": 0, "neighbors": [3,5,7], "hoppings": [0,0,0,t2,0,t3,0,t3]},
    {"U": 0, "neighbors": [0,4,6], "hoppings": [t2,0,0,0,t3,0,t3,0]},
    {"U": 0, "neighbors": [1,5,7], "hoppings": [0,t2,0,0,0,t3,0,t3]},
    {"U": 0, "neighbors": [2,4,6], "hoppings": [0,0,t2,0,t3,0,t3,0]}
]

model = model4site1d
N = len(model)
print(N)

4


In [36]:
def U():
    """
    Builds the interaction term: a_p^dagger a_q^dagger a_q a_p
    """    
    hamiltonian = ZERO()
    for i in range(N):
        hamiltonian += (model[i]["U"] / 4) * (sI() - sZ(i) - sZ(i+N) + sZ(i)*sZ(i+N))
        
    return hamiltonian

def t():
    """
    Builds the hopping term: a_p^dagger a_q
    """
    def op(s1, s2):
        """
        returns a PauliSum representing a_s1^daggar a_s2
        """    
        # for all sites in between site one (s1) and site two (s2), multiply by sigma z
        z = sI()
        for i in range(s1+1, s2):
            z *= PauliTerm('Z', i)

        return sX(s1) * z * sX(s2) + sY(s1) * z * sY(s2)
        
    hops = []
    for i in range(N):
        n = model[i]["neighbors"]
        for j in range(len(n)):
            # if a hopping is not already in the array i.e. don't add (3,0) if (0,3) is there
            if (i, n[j]) in hops or (n[j], i) in hops or i == n[j]:
                pass
            else:
                hops.append((i, n[j]))
                
    hamiltonian = ZERO()
    for hop in hops:
        # add hoppings for up and down spins
        hamiltonian += (-model[hop[0]]["hoppings"][hop[1]]) * op(hop[0], hop[1])
    for hop in hops:
        hamiltonian += (-model[hop[0]]["hoppings"][hop[1]]) * op(hop[0]+N, hop[1]+N)
    return hamiltonian * (1/2)

hamU = U()
hamt = t()

hamiltonian = hamU + hamt

In [37]:
# converting pyquil pauliSum to the form qulacs needs
ham_str = str(hamiltonian)

ham_str = ham_str.replace("*I", "[I]")
for p in ["X", "Y", "Z"]:
    for i in reversed(range(N*2)):            
        ham_str = ham_str.replace("*{}{}".format(p, i), "[{}{}]".format(p, i))
ham_str = ham_str.replace("][", " ")
ham_str = ham_str.replace(" + ", " +\n")

hamiltonian = create_observable_from_openfermion_text(ham_str)
print(ham_str)

(0.75+0j)[I] +
(-0.25+0j)[Z1] +
(-0.25+0j)[Z5] +
(0.25+0j)[Z1 Z5] +
(-0.25+0j)[Z2] +
(-0.25+0j)[Z6] +
(0.25+0j)[Z2 Z6] +
(-0.25+0j)[Z3] +
(-0.25+0j)[Z7] +
(0.25+0j)[Z3 Z7] +
(-0.5+0j)[X0 X1] +
(-0.5+0j)[Y0 Y1] +
(-0.5+0j)[X1 X2] +
(-0.5+0j)[Y1 Y2] +
(-0.5+0j)[X2 X3] +
(-0.5+0j)[Y2 Y3] +
(-0.5+0j)[X4 X5] +
(-0.5+0j)[Y4 Y5] +
(-0.5+0j)[X5 X6] +
(-0.5+0j)[Y5 Y6] +
(-0.5+0j)[X6 X7] +
(-0.5+0j)[Y6 Y7]


In [38]:
def U_ansatz(param, c):
    """
    Creates a circuit for e^{i \theta_U H_U}
    """
    for i in range(N):
        c.add_RZ_gate(i, model[i]["U"]*param/4)
        c.add_RZ_gate(i+N,  model[i]["U"]*param/4)
        c.add_CNOT_gate(i, i+N)
        c.add_RZ_gate(i+N, -model[i]["U"]*param/4)
        c.add_CNOT_gate(i, i+N)    
    return c

In [39]:
def t_ansatz(s1, s2, param, c):
    """
    Creates a circuit for e^{i \theta_t H_t} for horizontal and vertical hoppings
    """    
    for i in range(s1+1, s2):
        if (i == s2-1):
            c.add_CZ_gate(i, i+1)
        else:
            c.add_CNOT_gate(i, i+1)
    
    c.add_H_gate(s1)
    c.add_H_gate(s2)
    c.add_CNOT_gate(s1, s2)
    c.add_RZ_gate(s2, param)
    c.add_CNOT_gate(s1, s2)
    c.add_H_gate(s1)
    c.add_H_gate(s2)
    c.add_RX_gate(s1, -np.pi/2)
    c.add_RX_gate(s2, -np.pi/2)
    c.add_CNOT_gate(s1, s2)
    c.add_RZ_gate(s2, param)
    c.add_CNOT_gate(s1, s2)
    c.add_RX_gate(s1, -np.pi/2)
    c.add_RX_gate(s2, -np.pi/2)
    
    for i in reversed(range(s1+1, s2)):
        if (i == s2-1):
            c.add_CZ_gate(i, i+1)
        else:
            c.add_CNOT_gate(i, i+1)
    return c

In [40]:
# This is from the pyquil code, I am not sure what this cell is doing so I don't know how to convert it to qulacs
# The main issue was from how the model was set up in pyquil. Since I have to explicitly write the hamiltonian,
# I still am figuring out how to work with the neighbors and hopping terms

simplified_h_t =  np.array([[ 0.0 for i in range(N)] for j in range(N)])
for i in range(N):
    for j in model[i]["neighbors"]:
        simplified_h_t[i][j] = -1.0
# for i in range(N):
#     simplified_h_t[i][N-1-i] = -0.99
print(simplified_h_t)
w2, v2 = LA.eigh(simplified_h_t)

for i in range(N):
    print(w2[i], v2[:,i])

# convert to hopping direction matrix
for i in range(N):
    simplified_h_t[i][N-1-i] *= -1

[[ 0. -1.  0.  0.]
 [-1.  0. -1.  0.]
 [ 0. -1.  0. -1.]
 [ 0.  0. -1.  0.]]
-1.6180339887498947 [-0.37174803 -0.60150096 -0.60150096 -0.37174803]
-0.6180339887498946 [ 0.60150096  0.37174803 -0.37174803 -0.60150096]
0.6180339887498952 [ 0.60150096 -0.37174803 -0.37174803  0.60150096]
1.6180339887498942 [-0.37174803  0.60150096 -0.60150096  0.37174803]


In [41]:
# Q = get_mo(h1, eri, N)
Q = np.array(
[[ 0.49278338,  0.62174702 , 0.53789134,  0.28507557],
 [ 0.6142838,   0.20682541, -0.49528015, -0.57842565]]
 )

print(Q)
from openfermion.utils import slater_determinant_preparation_circuit

def prepare_slater_determinant(Q, c): 
    """
    Prepares the Slater determinant as described in https://arxiv.org/pdf/1711.05395.pdf
    
    Args:
        Q: The (N_f x N) matrix Q with orthonormal rows which describes the Slater determinant to be prepared.
        c: Program to append to
    Returns:
        c: A program that applies the sequence of Givens rotations returned 
            from slater_determinant_preparation_circuit
    """
    # Defining a controlled-RY gate    
    def CRY(control, target, angle):
        ry = q.gate.RY(target, angle*2)
        cry = q.gate.to_matrix_gate(ry)
        cry.add_control_qubit(control, 1) # add_control_qubit(control_index, control_with value) 
        return cry
    
    # Q is a (N_f x N) matrix
    # N = Q[0].size
    N_f = len(Q)
    
    givens = slater_determinant_preparation_circuit(Q)

    def givens_rotation(tups, spin, c):
        """
        Performs Givens rotations
        
        Args:
            tups: tuple containing Givens rotations to be performed together
            spin: 0 represents up spin, and 1 represents down spin
        Returns:
            p: A program that applies the Givens rotations in tups
        """
        for tup in tups:
            # where tup is (j, k, theta, phi)
            c.add_CNOT_gate(tup[1]+N*spin, tup[0]+N*spin)

            # controlled-RY
            c.add_gate(CRY(tup[0]+N*spin, tup[1]+N*spin, tup[2]))
            c.add_CNOT_gate(tup[1]+N*spin, tup[0]+N*spin)
            c.add_RZ_gate(tup[1]+N*spin, tup[3]) # all psis are zero, anyway       
        return c
    
    # Fill first N_f orbitals for each spin
    for i in range(N_f):
        c.add_X_gate(i)
        c.add_X_gate(i+N)

    # Perform Givens rotations for up and down spins
    for rot in givens:
        givens_rotation(rot, 0, c)
        givens_rotation(rot, 1, c)

    return c

c = QuantumCircuit(N*2) 
state = QuantumStateGpu(N*2)
c = prepare_slater_determinant(Q, c) #Construct quantum circuit
c.update_quantum_state(state) #Operate quantum circuit on state
print(slater_determinant_preparation_circuit(Q))
print(hamiltonian.get_expectation_value(state))

[[ 0.49278338  0.62174702  0.53789134  0.28507557]
 [ 0.6142838   0.20682541 -0.49528015 -0.57842565]]
[((1, 2, -1.0752190887651534, 0.0),), ((0, 1, -0.7762085052864728, 0.0), (2, 3, -0.8227952765689392, 0.0)), ((1, 2, -0.3856438378703364, 0.0),)]
-3.7779240783500505


In [42]:
hops = []
for j in range(N):
    n = model[j]["neighbors"]
    for k in range(len(n)):
        # if a hopping is not already in the array i.e. don't add (3,0) if (0,3) is there
        if (j, n[k]) in hops or (n[k], j) in hops or j == n[k]:
            pass
        else:
            hops.append((j, n[k]))
h_hops = []
v_hops = []
for hop in hops:
    if (simplified_h_t[hop[0]][hop[1]] < 0):
        h_hops.append(hop)
    else:
        v_hops.append(hop)

print(simplified_h_t)
print(h_hops)
print(v_hops)

def var_ansatz(params):
    """
    Prepares \sum_S [U_U(\theta_U / 2) U_h(\theta_h) U_v(\theta_v) U_U(\theta_U / 2)] | \Psi_I >
    Where | \Psi_I > is the Slater determinant for U = 0
    """
    S = 1
    c = QuantumCircuit(N*2)
    # c = prepare_slater_determinant(Q, c)
    
    for i in range(S):
        # building U_u
        c = U_ansatz(params[3*i], c)

        # building U_t, horizontal and vertical
        # up spin horizontal hops
        for hop in h_hops:
            c = t_ansatz(hop[0], hop[1], params[3*i+1], c)
        # down spin horizontal hops
        for hop in h_hops:
            c = t_ansatz(hop[0]+N, hop[1]+N, params[3*i+1], c)

        # up spin vertical hops
        for hop in v_hops:
            c = t_ansatz(hop[0], hop[1], params[3*i+2], c)
        # down spin vertical hops
        for hop in v_hops:
            c = t_ansatz(hop[0]+N, hop[1]+N, params[3*i+2], c)  
    
        # building U_u
        c = U_ansatz(params[3*i], c)
        
    return c

[[ 0. -1.  0. -0.]
 [-1.  0.  1.  0.]
 [ 0.  1.  0. -1.]
 [-0.  0. -1.  0.]]
[(0, 1), (2, 3)]
[(1, 2)]


In [43]:
from scipy.optimize import minimize

def expectation(x, circuits):
    """
    Expectation value for parameters, x
    """
    state = QuantumStateGpu(N*2)
    for c in circuits:
        c.update_quantum_state(state)
    c_var = var_ansatz(x)
    c_var.update_quantum_state(state)
    return hamiltonian.get_expectation_value(state)

def scipy_minimize(x, hamiltonian, circuits):
    """
    Minimizes expectation value using parameters found in greedy search as starting point
    Powell's conjugate direction method
    """    
    return minimize(expectation, x, args=(circuits), method='powell')

def greedy_noisy_search(initial_state, hamiltonian, circuits):
    """
    Slightly perturb the values of the points, accepting whenever this results in a lower energy
    Total of 150 evaulations of the energy
    Change step size after 30 evaluations based on number of acceptances in previous trial group
    """
    params = initial_state
    min_energy = expectation(params, circuits)

    def random_point(dim, step):
        """
        Generates a random point on the perimeter of a circle with radius, step
        """
        coords = [random.gauss(0, 1) for i in range(dim)]
        norm = math.sqrt(sum([i**2 for i in coords]))
        coords = [(i / norm) * step for i in coords]
        return coords

    step = 0.5
    acceptances = 0

    # Five groups of 30 trials
    for i in range(5):
        acceptances = 0
        for j in range(30):
            
            # Slightly perturb the values of the points
            coords = random_point(len(initial_state), step)
            temp_params = [sum(x) for x in zip(params, coords)]
            
            # Calculate expectation value with new parameters
            temp_energy = expectation(temp_params, circuits)
            
            # Greedily accept parameters that result in lower energy
            if (temp_energy < min_energy):
                min_energy = temp_energy
                params = temp_params
                acceptances += 1
        # Update step size
        step *= (acceptances / 15)

    return {'x': params, 'energy': min_energy, 'step': step}

def global_variational(hamiltonian, circuits):
    """
    Complete a round of greedy and Powell's optimization for six randomly chosen points
    Further optimizes the best point
    """
    points = [np.random.rand(3)*0.2 for i in range(3)]
    greedy_results = [greedy_noisy_search(x, hamiltonian, circuits) for x in points]
    results = [scipy_minimize(i['x'], hamiltonian, circuits) for i in greedy_results]
    res = min(results, key=lambda x:x.fun)
    
    # For the chosen point, we continue optimizing until we cannot find improvement
    res_copy = res
    energy = res_copy.fun
    while (True):
        result = greedy_noisy_search(res_copy.x, hamiltonian, circuits)
        temp_res = scipy_minimize(result['x'], hamiltonian, circuits)
        temp_energy = temp_res.fun
        print(temp_energy)

        # Optimizing will usually find an energy ~1e-6 lower
        # Stop it eventually
        tolerance = 0.001
        if (abs(temp_energy - energy) > tolerance):
            res_copy = temp_res
            energy = temp_energy
        else:
            break
    return res_copy

In [47]:
def annealed_variational():
    S = 3
    circuits = []
    circuits.append(prepare_slater_determinant(Q, c))
    params = []
    for i in range(S+1):
        hamU = (1+(i/S))*U()
        hamt = t()
        hamiltonian = hamU + hamt
        print(hamiltonian)
        res = global_variational(hamiltonian, circuits)
        
        params.append(res.x)
        circuits.append(var_ansatz(res.x))
        
    return (params, res)

In [48]:
params, res = annealed_variational()
print(res)

(0.75+0j)*I + (-0.25+0j)*Z1 + (-0.25+0j)*Z5 + (0.25+0j)*Z1*Z5 + (-0.25+0j)*Z2 + (-0.25+0j)*Z6 + (0.25+0j)*Z2*Z6 + (-0.25+0j)*Z3 + (-0.25+0j)*Z7 + (0.25+0j)*Z3*Z7 + (-0.5+0j)*X0*X1 + (-0.5+0j)*Y0*Y1 + (-0.5+0j)*X1*X2 + (-0.5+0j)*Y1*Y2 + (-0.5+0j)*X2*X3 + (-0.5+0j)*Y2*Y3 + (-0.5+0j)*X4*X5 + (-0.5+0j)*Y4*Y5 + (-0.5+0j)*X5*X6 + (-0.5+0j)*Y5*Y6 + (-0.5+0j)*X6*X7 + (-0.5+0j)*Y6*Y7
-0.9605326740189825
(1+0j)*I + (-0.3333333333333333+0j)*Z1 + (-0.3333333333333333+0j)*Z5 + (0.3333333333333333+0j)*Z1*Z5 + (-0.3333333333333333+0j)*Z2 + (-0.3333333333333333+0j)*Z6 + (0.3333333333333333+0j)*Z2*Z6 + (-0.3333333333333333+0j)*Z3 + (-0.3333333333333333+0j)*Z7 + (0.3333333333333333+0j)*Z3*Z7 + (-0.5+0j)*X0*X1 + (-0.5+0j)*Y0*Y1 + (-0.5+0j)*X1*X2 + (-0.5+0j)*Y1*Y2 + (-0.5+0j)*X2*X3 + (-0.5+0j)*Y2*Y3 + (-0.5+0j)*X4*X5 + (-0.5+0j)*Y4*Y5 + (-0.5+0j)*X5*X6 + (-0.5+0j)*Y5*Y6 + (-0.5+0j)*X6*X7 + (-0.5+0j)*Y6*Y7
-0.5578237476481135
(1.25+0j)*I + (-0.41666666666666663+0j)*Z1 + (-0.41666666666666663+0j)*Z5 + (0.41